In [ ]:
%cd /home/parthgandhi/TradeBot

In [ ]:
import polars as pl
import polars.selectors as cs
from src.config.storage_layout import StorageLayout
from src.config.market import Market
from src.config.exchange import Exchange

In [ ]:
END_DATE = "2025-12-26"
RS_RATING = 70

In [ ]:
#  rclone sync /home/parthgandhi/TradeBot/storage/data/IND/ChartsMaze gdrive:Backup/SwingTrade/ChartsMaze/RS --progress

# Industry Analysis

In [ ]:
analysis_path = StorageLayout.analysis_dir(
    run_date=END_DATE, market=Market.INDIA_EQUITIES, exchange=Exchange.NSE
)

res = pl.scan_csv(analysis_path / "overall_filter_result.csv").filter(
    pl.col("rs_rating") >= RS_RATING
)

In [ ]:
industry_analysis = (
    res.lazy()
    .group_by("sector_cmaze", "basic_industry_cmaze")
    .agg(
        [pl.col("symbol").count().alias("symbols_count")]
        + [
            (
                (pl.col(f"{i}_mo_rtr_pct") * pl.col("market_cap_cr")).sum()
                / pl.col("market_cap_cr_cmaze").sum()
            )
            .round(2)
            .alias(f"{i}_mo_avg_weight_rtr_pct")
            for i in [1, 3]
        ]
        + [
            (
                (pl.col("rs_rating") * pl.col("market_cap_cr_cmaze")).sum()
                / pl.col("market_cap_cr_cmaze").sum()
            )
            .round(2)
            .alias("rs_rating_avg_weight")
        ]
    )
    .fill_nan(None)
    .with_columns(
        pl.mean_horizontal(
            cs.exclude("sector_cmaze", "basic_industry_cmaze", "symbols_count")
        )
        .round(2)
        .alias("industry_score"),
        (pl.col("1_mo_avg_weight_rtr_pct") * 100 / pl.col("3_mo_avg_weight_rtr_pct"))
        .round()
        .alias("1_by_3_rtr"),
    )
    .sort("symbols_count", descending=True)
    .collect()
)

industry_analysis.write_csv(analysis_path / "industry_analysis.csv")

# RS >= 70 & Possible Upcoming Centers

In [ ]:
industry_analysis.filter(
    (pl.col("rs_rating_avg_weight") >= 70)
    & (pl.col("1_mo_avg_weight_rtr_pct") >= pl.col("3_mo_avg_weight_rtr_pct"))
).sort("industry_score", descending=True)

In [ ]:
industry_analysis.filter((pl.col("rs_rating_avg_weight") >= 70)).sort(
    "industry_score", descending=True
).filter(pl.col("symbols_count") > 1)